# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# %load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

# pgconfig = {
#     'host': 'db',
#     'port': os.environ['PG_PORT'],
#     'database': os.environ['PG_DATABASE'],
#     'user': os.environ['PG_USER'],
#     'password': os.environ['PG_PASSWORD'],
# }
# dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# # pd.read_sql用のコネクタ
# conn = psycopg2.connect(**pgconfig)
# # pd.to_sql用のcreate engine
# engine = create_engine(dsl)
# # MagicコマンドでSQLを書くための設定
# %sql $dsl

# df_customer = pd.read_sql(sql='select * from customer', con=conn)
# df_category = pd.read_sql(sql='select * from category', con=conn)
# df_product = pd.read_sql(sql='select * from product', con=conn)
# df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
# df_store = pd.read_sql(sql='select * from store', con=conn)
# df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [5]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [7]:
df_geocode.head()

,postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
0,060-0000,北海道,札幌市中央区,NaN,NaN,NaN,北海道札幌市中央区,141.34103,43.05513
1,064-0941,北海道,札幌市中央区,旭ケ丘,NaN,NaN,北海道札幌市中央区旭ケ丘,141.31972,43.04223
2,060-0042,北海道,札幌市中央区,大通西,NaN,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102
3,060-0042,北海道,札幌市中央区,大通西,NaN,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080
4,060-0042,北海道,札幌市中央区,大通西,NaN,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086


# 演習問題

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [8]:
# 組み込みのroundはNaNでエラーとなるが、numpy.roundはエラーとならない
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.round(x / 0.7))
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,4,401,40101,198.0,149.0,213.0,0.300469
1,P040101002,4,401,40101,218.0,164.0,234.0,0.299145
2,P040101003,4,401,40101,230.0,173.0,247.0,0.299595
3,P040101004,4,401,40101,248.0,186.0,266.0,0.300752
4,P040101005,4,401,40101,268.0,201.0,287.0,0.299652
5,P040101006,4,401,40101,298.0,224.0,320.0,0.300000
6,P040101007,4,401,40101,338.0,254.0,363.0,0.300275
7,P040101008,4,401,40101,420.0,315.0,450.0,0.300000
8,P040101009,4,401,40101,498.0,374.0,534.0,0.299625
9,P040101010,4,401,40101,580.0,435.0,621.0,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [9]:
# math.ceilはNaNでエラーとなるが、numpy.ceilはエラーとならない
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.ceil(x / 0.7))
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,4,401,40101,198.0,149.0,213.0,0.300469
1,P040101002,4,401,40101,218.0,164.0,235.0,0.302128
2,P040101003,4,401,40101,230.0,173.0,248.0,0.302419
3,P040101004,4,401,40101,248.0,186.0,266.0,0.300752
4,P040101005,4,401,40101,268.0,201.0,288.0,0.302083
5,P040101006,4,401,40101,298.0,224.0,320.0,0.300000
6,P040101007,4,401,40101,338.0,254.0,363.0,0.300275
7,P040101008,4,401,40101,420.0,315.0,451.0,0.301552
8,P040101009,4,401,40101,498.0,374.0,535.0,0.300935
9,P040101010,4,401,40101,580.0,435.0,622.0,0.300643


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [10]:
# math.floorはNaNでエラーとなるが、numpy.floorはエラーとならない
df_tmp = df_product.copy()
df_tmp['price_tax'] = df_tmp['unit_price'].apply(lambda x: np.floor(x * 1.1))
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,price_tax
0,P040101001,4,401,40101,198.0,149.0,217.0
1,P040101002,4,401,40101,218.0,164.0,239.0
2,P040101003,4,401,40101,230.0,173.0,253.0
3,P040101004,4,401,40101,248.0,186.0,272.0
4,P040101005,4,401,40101,268.0,201.0,294.0
5,P040101006,4,401,40101,298.0,224.0,327.0
6,P040101007,4,401,40101,338.0,254.0,371.0
7,P040101008,4,401,40101,420.0,315.0,462.0
8,P040101009,4,401,40101,498.0,374.0,547.0
9,P040101010,4,401,40101,580.0,435.0,638.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [11]:
# コード例1
df_tmp_1 = pd.merge(df_receipt, df_product, 
                    how='inner', on='product_cd').groupby('customer_id').agg({'amount':'sum'}).reset_index()

df_tmp_2 = pd.merge(df_receipt, df_product.query('category_major_cd == "07"'), 
                    how='inner', on='product_cd').groupby('customer_id').agg({'amount':'sum'}).reset_index()

df_tmp_3 = pd.merge(df_tmp_1, df_tmp_2, how='inner', on='customer_id')
df_tmp_3['rate_07'] = df_tmp_3['amount_y'] / df_tmp_3['amount_x']
df_tmp_3.head(10)

,customer_id,amount_x,amount_y,rate_07
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


In [12]:
# コード例2
df_temp = df_receipt.merge(df_product, how='left', on='product_cd').groupby(['customer_id', 'category_major_cd'])['amount'].sum().unstack()
df_temp.head()

category_major_cd,4,5,6,7,8,9
customer_id,,,,,,
CS001113000004,NaN,NaN,NaN,1298.0,NaN,NaN
CS001114000005,NaN,40.0,NaN,486.0,100.0,NaN
CS001115000010,NaN,NaN,NaN,2694.0,NaN,350.0
CS001205000004,100.0,128.0,286.0,346.0,368.0,760.0
CS001205000006,635.0,60.0,198.0,2004.0,80.0,360.0


In [13]:
df_temp = df_temp[df_temp[7] > 0] # 修正した
df_temp['sum'] = df_temp.sum(axis=1)
df_temp['07_rate'] = df_temp[7] / df_temp['sum'] # 修正した
df_temp.head(10)

category_major_cd,4,5,6,7,8,9,sum,07_rate
customer_id,,,,,,,,
CS001113000004,NaN,NaN,NaN,1298.0,NaN,NaN,1298.0,1.000000
CS001114000005,NaN,40.0,NaN,486.0,100.0,NaN,626.0,0.776358
CS001115000010,NaN,NaN,NaN,2694.0,NaN,350.0,3044.0,0.885020
CS001205000004,100.0,128.0,286.0,346.0,368.0,760.0,1988.0,0.174044
CS001205000006,635.0,60.0,198.0,2004.0,80.0,360.0,3337.0,0.600539
CS001212000027,248.0,NaN,NaN,200.0,NaN,NaN,448.0,0.446429
CS001212000031,NaN,NaN,NaN,296.0,NaN,NaN,296.0,1.000000
CS001212000046,NaN,NaN,NaN,108.0,NaN,120.0,228.0,0.473684
CS001212000070,NaN,NaN,148.0,308.0,NaN,NaN,456.0,0.675439


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [14]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp = df_tmp.drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'].astype('str')) # 修正
df_tmp['elapsed_date'] = df_tmp['sales_ymd'] - df_tmp['application_date']
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
4,CS006214000001,2018-10-28,2015-02-01,1365 days
7,CS006214000001,2019-09-08,2015-02-01,1680 days
8,CS006214000001,2018-01-31,2015-02-01,1095 days
9,CS006214000001,2017-07-05,2015-02-01,885 days
10,CS006214000001,2018-11-10,2015-02-01,1378 days
12,CS006214000001,2019-04-10,2015-02-01,1529 days
15,CS006214000001,2019-06-01,2015-02-01,1581 days
